In [1]:
import numpy as np
import csv
import time

np.random.seed(1234)
def randomize(): np.random.seed(time.time())

In [30]:
# Hyperparameters

RND_MEAN = 0
RND_STD = 0.0030

LEARNING_RATE = 0.001

In [59]:
def abalone_exec(epoch_count=10, mb_size=10, report=1):
    load_abalone_dataset()
    init_model()
    train_and_test(epoch_count, mb_size, report)

In [62]:
def load_abalone_dataset():
    with open('./DataSets/abalone.data.csv') as csvfile:
        csvreader = csv.reader(csvfile)
#         next(csvreader, None)  # ignore header
        rows = []
        for row in csvreader:
            rows.append(row)
            
            
    global data, input_cnt, output_cnt
    input_cnt, output_cnt = 10, 1
    data = np.zeros([len(rows), input_cnt+output_cnt])
    
    for n , row in enumerate(rows):
        if row[0] == 'I': data[n,0] = 1
        if row[0] == 'M': data[n,1] = 1
        if row[0] == 'F': data[n,2] = 1
        data[n, 3:] = row[1:]

In [64]:
def init_model():
    global weight, bias, input_cnt, output_cnt
    weight = np.random.normal(RND_MEAN, RND_STD, [input_cnt, output_cnt])
    bias = np.zeros([output_cnt])

In [8]:
def train_and_test(epoch_count, mb_size, report):
    step_count = arrange_data(mb_size)
    test_x, test_y = get_test_data()
    
    for epoch in range(epoch_count):
        losses, accs = [], []
        
        for n in range(step_count):
            train_x, train_y = get_train_data(mb_size, n)
            loss, acc = run_train(train_x, train_y)
            losses.append(loss)
            accs.append(acc)
            
        if report > 0 and (epoch+1) % report == 0:
            acc = run_test(test_x, test_y)
            print('Epoch {}: loss={:5.3f}, accuracy={:5.3f}/{:5.3f}]'.format(epoch+1, np.mean(losses), np.mean(accs), acc))
            
    final_acc = run_test(test_x, test_y)
    print('\nFinal Test: final accuracy = {:5.3f}'.format(final_acc))

In [68]:
def arrange_data(mb_size):
    global data, shuffle_map, test_begin_idx
    shuffle_map = np.arange(data.shape[0])
    np.random.shuffle(shuffle_map)
    step_count = int(data.shape[0] * 0.8) // mb_size
    test_begin_idx = step_count * mb_size
    return step_count

def get_test_data():
    global data, shuffle_map, test_begin_dix, output_cnt
    test_data = data[shuffle_map[test_begin_idx:]]
    return test_data[:, :-output_cnt], test_data[:, -output_cnt:]

def get_train_data(mb_size, nth):
    global data, shuffle_map, test_begin_idx, output_cnt
    if nth == 0:
        np.random.shuffle(shuffle_map[:test_begin_idx])
    train_data = data[shuffle_map[mb_size*nth:mb_size*(nth+1)]]
    return train_data[:, :-output_cnt], train_data[:, -output_cnt:]

In [17]:
def run_train(x,y):
    output, aux_nn = forward_neuralnet(x)
    loss, aux_pp = forward_postproc(output, y)
    accuracy = eval_accuracy(output, y)
    
    G_loss = 1.0
    G_output = backprop_postproc(G_loss, aux_pp)
    backprop_neuralnet(G_output, aux_nn)
    
    return loss, accuracy

def run_test(x,y):
    output, _ = forward_neuralnet(x)
    accuracy = eval_accuracy(output, y)
    return accuracy

In [48]:
def forward_neuralnet(x):
    global weight, bias
    output = np.matmul(x, weight) + bias
    return output, x

def backprop_neuralnet(G_output, x):
    global weight, bias
    g_output_w = x.transpose()
    
    G_w = np.matmul(g_output_w, G_output)
    G_b = np.sum(G_output, axis=0)
    
    weight -= LEARNING_RATE * G_w
    bias -= LEARNING_RATE * G_b

In [54]:
def forward_postproc(output, y):
    diff = output - y
    square = np.square(diff)
    loss = np.mean(square)
    return loss, diff

def backprop_postproc(G_loss, diff):
    shape = diff.shape
    
    g_loss_square = np.ones(shape) / np.prod(shape) # square에 대한 L의 기울기
    g_square_diff = 2 * diff # diff에 대한 square의 기울기
    g_diff_output = 1 # output에 대한 diff의 기울기
    
    G_square = g_loss_square * G_loss
    G_diff = g_square_diff * G_square
    G_output = g_diff_output * G_diff
    
    return G_output

In [55]:
def eval_accuracy(output, y):
    mdiff = np.mean(np.abs((output-y)/y))
    return 1 - mdiff

In [69]:
abalone_exec()

Epoch 1: loss=34.200, accuracy=0.560/0.816]
Epoch 2: loss=8.418, accuracy=0.818/0.820]
Epoch 3: loss=7.735, accuracy=0.810/0.815]
Epoch 4: loss=7.623, accuracy=0.808/0.813]
Epoch 5: loss=7.542, accuracy=0.807/0.814]
Epoch 6: loss=7.472, accuracy=0.808/0.814]
Epoch 7: loss=7.412, accuracy=0.808/0.813]
Epoch 8: loss=7.360, accuracy=0.807/0.815]
Epoch 9: loss=7.314, accuracy=0.808/0.813]
Epoch 10: loss=7.271, accuracy=0.806/0.816]

Final Test: final accuracy = 0.816


In [70]:
print(weight)
print(bias)

[[1.01316282]
 [1.51926124]
 [1.63476282]
 [2.03948247]
 [1.61877545]
 [0.59738261]
 [2.35822866]
 [0.51970025]
 [0.47095111]
 [1.02488524]]
[4.17532529]


In [71]:
LEARNING_RATE = 0.1
abalone_exec(epoch_count=100, mb_size=100, report=20)

Epoch 20: loss=5.657, accuracy=0.826/0.812]
Epoch 40: loss=5.145, accuracy=0.835/0.834]
Epoch 60: loss=4.961, accuracy=0.838/0.836]
Epoch 80: loss=4.870, accuracy=0.839/0.822]
Epoch 100: loss=4.850, accuracy=0.840/0.836]

Final Test: final accuracy = 0.836
